In [2]:
from configparser import ConfigParser
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import time

parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET), requests_timeout=20, retries=5)

In [ ]:
# Deprecated

# Get albums and tracks from artists.csv
input = pd.read_csv('artists.csv', header = None)
artist_ids = []
for link in input[0]:
    
    artist_ids.append(link[32:54])

albums = []
album_track_ids = []
album_ids = []
artist_uri = 'spotify:artist:' + artist_ids[0]

# Add album ids to album_ids list
results = sp.artist_albums(artist_uri, album_type='album')
albums.extend(results['items'])
while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])
for album in albums:
    album_ids.append(album['id'])

# Get track list
all_tracks = []
for id in album_ids:

    track_ids = []

    results = sp.album_tracks(id)
    tracks = results['items']
    all_tracks.extend(tracks)
    for track in tracks:
        track_ids.append(track['id'])

    album_track_ids.append(track_ids)


assert len(album_track_ids) == len(albums)

len(albums)

# Maunal dataset size reduction
# albums = albums[0:2]
album_track_ids = album_track_ids[0:2]
album_track_ids

In [ ]:
# Test block

# sp.audio_features('2TmF1VCPueHV2N65MgrKnz')
# print(len(sp.audio_analysis('0Kbvz9ZjihrcE1Vd1dDCZh')['segments']))
print(len(sp.audio_analysis('2TmF1VCPueHV2N65MgrKnz')['segments']))

In [ ]:
# Section dataset generation

# Set up data
headings = ['start',
'duration',
'confidence',
'loudness',
'tempo',
'tempo_confidence',
'key',
'key_confidence',
'mode',
'mode_confidence',
'time_signature',
'time_signature_confidence']
section_data = pd.DataFrame(columns=headings)
input_data = pd.read_csv("./data/data.csv")

# Create DataFrames of features from track ids
i = 0
for row in range(input_data.shape[0]):
    track_sections = sp.audio_analysis(input_data.iloc[i,12])['sections']
    for results in track_sections:
        if results: 
            features = results
        features_matrix = pd.DataFrame.from_records(features, index=[0])
        
        # Add additional data features from input data
        features_matrix['track_title'] = input_data.iloc[i,15]
        features_matrix['album_title'] = input_data.iloc[i,16]
        features_matrix['album_artist'] = input_data.iloc[i,17]

        section_data = section_data.append(features_matrix)

    i += 1

section_data

In [ ]:
section_data.to_csv('./data/sections.csv')
section_data

In [ ]:
input_data = pd.read_csv("./data/data.csv")

lengths = []
for i in range(500):
    lengths.append(len(sp.audio_analysis(input_data.iloc[i,12])['segments']))
np.mean(lengths)

In [ ]:
# Segment dataset generation

# Set up data
headings = ['start',
            'duration',
            'confidence',
            'loudness_start',
            'loudness_max_time',
            'loudness_max',
            'loudness_end',
            'track_title',
            'album_title',
            'album_artist',
            'order']

input_data = pd.read_csv("./data/data.csv")
segment_data = pd.DataFrame(columns = headings)

# Create DataFrames of features from track ids
i = 4543
pitches = []
timbres = []
for row in range(4543,input_data.shape[0]):
    # if i%200 == 0:
    #     time.wait(60)
    print(str(i) + ' ' + input_data.iloc[i,15])
    track_segments = sp.audio_analysis(input_data.iloc[i,12])['segments']

    # Extract pitch and timbre data from track
    for results in track_segments:
        pitches.append(results.get('pitches'))
        del results['pitches']
        timbres.append(results.get('timbre'))
        del results['timbre']

    # Add track segment data to dataframe
    for results in track_segments:

        if results: 
            features = results

        segment_features_matrix = pd.DataFrame.from_records(features, index=[0])

        # Add additional data features from input data
        segment_features_matrix['track_title'] = input_data.iloc[i,15]
        segment_features_matrix['album_title'] = input_data.iloc[i,16]
        segment_features_matrix['album_artist'] = input_data.iloc[i,17]
        segment_features_matrix['order'] = input_data.iloc[i,20]

        segment_data = segment_data.append(segment_features_matrix)

    i += 1

# # Delete empty rows of dataframe
# for row in range(segment_data.shape[0]-1,0,-1):
#     if segment_data.iloc[row,10] == 0:
#         segment_data.drop(row, inplace=True)

# Add pitch and timbre data
segment_data['pitches'] = pitches
segment_data['timbres'] = timbres
segment_data.to_csv('./data/segments_end.csv')

segment_data

In [ ]:
# Additional data handling for stoppages and errors in previous block

# Set pitches and timbres to same size as segment_data (semi-manual)
pitches_temp = pitches[0:357394]
timbres_temp = timbres[0:357394]

segment_data['pitches'] = pitches_temp
segment_data['timbres'] = timbres_temp

# segment_data['pitches'] = pitches
# segment_data['timbres'] = timbres

segment_data.to_csv('./data/segments4542.csv')
segment_data

In [ ]:
# Combine segments dataset files

segments270 = pd.read_csv("./data/segments270.csv")
segments506 = pd.read_csv("./data/segments506.csv")
segments522 = pd.read_csv("./data/segments522.csv")
segments533 = pd.read_csv("./data/segments533.csv")
segments549 = pd.read_csv("./data/segments549.csv")
segments643 = pd.read_csv("./data/segments643.csv")
segments664 = pd.read_csv("./data/segments664.csv")
segments676 = pd.read_csv("./data/segments676.csv")
segments688 = pd.read_csv("./data/segments688.csv")
segments752 = pd.read_csv("./data/segments752.csv")
segments763 = pd.read_csv("./data/segments763.csv")
segments1786 = pd.read_csv("./data/segments1786.csv")
segments2302 = pd.read_csv("./data/segments2302.csv")
segments2794 = pd.read_csv("./data/segments2794.csv")
segments3222 = pd.read_csv("./data/segments3222.csv")
segments3659 = pd.read_csv("./data/segments3659.csv")
segments4123 = pd.read_csv("./data/segments4123.csv")
segments4542 = pd.read_csv("./data/segments4542.csv")
segments_end = pd.read_csv("./data/segments_end.csv")

all_segments = segments270
all_segments = all_segments.append(segments506)
all_segments = all_segments.append(segments522)
all_segments = all_segments.append(segments533)
all_segments = all_segments.append(segments549)
all_segments = all_segments.append(segments643)
all_segments = all_segments.append(segments664)
all_segments = all_segments.append(segments676)
all_segments = all_segments.append(segments688)
all_segments = all_segments.append(segments752)
all_segments = all_segments.append(segments763)
all_segments = all_segments.append(segments1786)
all_segments = all_segments.append(segments2302)
all_segments = all_segments.append(segments2794)
all_segments = all_segments.append(segments3222)
all_segments = all_segments.append(segments3659)
all_segments = all_segments.append(segments4123)
all_segments = all_segments.append(segments4542)
all_segments = all_segments.append(segments_end)

all_segments.to_csv('./data/all_segments.csv')
all_segments

In [29]:
# Add track ids
all_segments = pd.read_csv('./data/all_segments.csv')
input_data = pd.read_csv('./data/data.csv')

# Remove irrelevant columns
columnsToRemove = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1']
all_segments.drop(columns = columnsToRemove, axis = 1, inplace = True)

all_segments


,start,duration,confidence,loudness_start,loudness_max_time,loudness_max,loudness_end,track_title,album_title,album_artist,order,pitches,timbres,track_ids
0,0.00000,1.01583,0.000,-60.000,0.00000,-60.000,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,0.076923,"[0.202, 0.246, 0.215, 0.198, 0.407, 1.0, 0.246...","[0.0, 171.13, 9.469, -28.48, 57.491, -50.067, ...",0dEIca2nhcxDUV8C5QkPYb
1,1.01583,0.31596,1.000,-60.000,0.07260,-9.859,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,0.076923,"[0.169, 0.14, 0.057, 0.42, 1.0, 0.139, 0.033, ...","[35.458, 55.906, 69.742, -166.536, -3.252, 103...",0dEIca2nhcxDUV8C5QkPYb
2,1.33179,0.20694,0.630,-16.198,0.00584,-9.356,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,0.076923,"[0.559, 0.518, 0.394, 0.09, 0.082, 0.069, 0.30...","[47.965, 57.123, -5.636, -1.102, -15.098, -44....",0dEIca2nhcxDUV8C5QkPYb
3,1.53873,0.65542,0.721,-14.680,0.04780,-6.658,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,0.076923,"[0.156, 0.123, 0.097, 0.064, 0.292, 0.094, 0.0...","[49.72, 76.665, 1.289, -54.631, -18.938, 1.954...",0dEIca2nhcxDUV8C5QkPYb
4,2.19415,0.13379,0.260,-11.337,0.05143,-8.078,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,0.076923,"[0.222, 0.188, 0.054, 0.109, 0.452, 0.159, 0.4...","[50.446, 81.393, 18.975, -21.829, -16.591, -18...",0dEIca2nhcxDUV8C5QkPYb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4577946,418.23206,0.15161,1.000,-44.647,0.02458,-13.523,0.0,Pools - Roosevelt Remix,ZABA (Deluxe),Glass Animals,1.000000,"[1.0, 0.208, 0.092, 0.099, 0.098, 0.086, 0.083...","[36.131, 115.536, -81.509, 91.189, 66.389, 46....",1SARf3ugbFU6edbFEmNcgN
4577947,418.38367,0.12122,0.446,-35.956,0.01061,-24.762,0.0,Pools - Roosevelt Remix,ZABA (Deluxe),Glass Animals,1.000000,"[1.0, 0.158, 0.029, 0.021, 0.017, 0.032, 0.028...","[27.024, 49.855, -83.231, 76.299, 44.064, -83....",1SARf3ugbFU6edbFEmNcgN
4577948,418.50490,0.27274,1.000,-43.044,0.02553,-7.520,0.0,Pools - Roosevelt Remix,ZABA (Deluxe),Glass Animals,1.000000,"[0.942, 1.0, 0.511, 0.387, 0.299, 0.242, 0.177...","[37.884, 56.306, -138.516, 147.892, 5.411, 27....",1SARf3ugbFU6edbFEmNcgN
4577949,418.77765,0.14535,1.000,-44.700,0.02313,-22.244,0.0,Pools - Roosevelt Remix,ZABA (Deluxe),Glass Animals,1.000000,"[1.0, 0.188, 0.078, 0.153, 0.135, 0.063, 0.036...","[28.183, 84.257, -91.961, 103.347, 42.71, -34....",1SARf3ugbFU6edbFEmNcgN


In [40]:
# Add track ids

input_data = pd.read_csv('./data/data.csv')

input_counter = 0
segment_counter = 0
track_ids = []

for row in range(all_segments.shape[0]):
    if all_segments.iloc[segment_counter, 0] == 0:
        current_track_id = input_data.iloc[input_counter, 12]
        input_counter += 1

    track_ids.append(current_track_id)
    segment_counter += 1

all_segments['track_ids'] = track_ids
all_segments.to_csv('./data/all_segments.csv')
all_segments

,start,duration,confidence,loudness_start,loudness_max_time,loudness_max,loudness_end,track_title,album_title,album_artist,order,pitches,timbres,track_ids
0,0.00000,1.01583,0.000,-60.000,0.00000,-60.000,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,0.076923,"[0.202, 0.246, 0.215, 0.198, 0.407, 1.0, 0.246...","[0.0, 171.13, 9.469, -28.48, 57.491, -50.067, ...",0dEIca2nhcxDUV8C5QkPYb
1,1.01583,0.31596,1.000,-60.000,0.07260,-9.859,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,0.076923,"[0.169, 0.14, 0.057, 0.42, 1.0, 0.139, 0.033, ...","[35.458, 55.906, 69.742, -166.536, -3.252, 103...",0dEIca2nhcxDUV8C5QkPYb
2,1.33179,0.20694,0.630,-16.198,0.00584,-9.356,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,0.076923,"[0.559, 0.518, 0.394, 0.09, 0.082, 0.069, 0.30...","[47.965, 57.123, -5.636, -1.102, -15.098, -44....",0dEIca2nhcxDUV8C5QkPYb
3,1.53873,0.65542,0.721,-14.680,0.04780,-6.658,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,0.076923,"[0.156, 0.123, 0.097, 0.064, 0.292, 0.094, 0.0...","[49.72, 76.665, 1.289, -54.631, -18.938, 1.954...",0dEIca2nhcxDUV8C5QkPYb
4,2.19415,0.13379,0.260,-11.337,0.05143,-8.078,0.0,Give Life Back to Music,Random Access Memories,Daft Punk,0.076923,"[0.222, 0.188, 0.054, 0.109, 0.452, 0.159, 0.4...","[50.446, 81.393, 18.975, -21.829, -16.591, -18...",0dEIca2nhcxDUV8C5QkPYb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4577946,418.23206,0.15161,1.000,-44.647,0.02458,-13.523,0.0,Pools - Roosevelt Remix,ZABA (Deluxe),Glass Animals,1.000000,"[1.0, 0.208, 0.092, 0.099, 0.098, 0.086, 0.083...","[36.131, 115.536, -81.509, 91.189, 66.389, 46....",0zZdUmJhefdt9zasfN4nfw
4577947,418.38367,0.12122,0.446,-35.956,0.01061,-24.762,0.0,Pools - Roosevelt Remix,ZABA (Deluxe),Glass Animals,1.000000,"[1.0, 0.158, 0.029, 0.021, 0.017, 0.032, 0.028...","[27.024, 49.855, -83.231, 76.299, 44.064, -83....",0zZdUmJhefdt9zasfN4nfw
4577948,418.50490,0.27274,1.000,-43.044,0.02553,-7.520,0.0,Pools - Roosevelt Remix,ZABA (Deluxe),Glass Animals,1.000000,"[0.942, 1.0, 0.511, 0.387, 0.299, 0.242, 0.177...","[37.884, 56.306, -138.516, 147.892, 5.411, 27....",0zZdUmJhefdt9zasfN4nfw
4577949,418.77765,0.14535,1.000,-44.700,0.02313,-22.244,0.0,Pools - Roosevelt Remix,ZABA (Deluxe),Glass Animals,1.000000,"[1.0, 0.188, 0.078, 0.153, 0.135, 0.063, 0.036...","[28.183, 84.257, -91.961, 103.347, 42.71, -34....",0zZdUmJhefdt9zasfN4nfw
